# Как понять, что в коде Python можно что-то улучшить?

## 1. Измеряйте производительность

Измеряйте производительность блоков кода с помощью модулей time, [timeit](https://docs.python.org/3/library/timeit.html) и инструментов профилирования.

In [2]:
import timeit
timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)

0.16660843200224917

In [3]:
timeit.timeit('"-".join([str(n) for n in range(100)])', number=10000)

0.16614718100026948

In [4]:
timeit.timeit('"-".join(map(str, range(100)))', number=10000)

0.12677264800004195

В Jupyter для этого есть magic lines `time`, `timeit`, `prun` и работающая вместе со сторонней  библиотекой `line_profiler` команда `lprun`:

In [5]:
%timeit pass

8.13 ns ± 0.254 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [7]:
%time sum(range(10**5))

CPU times: user 7.41 ms, sys: 0 ns, total: 7.41 ms
Wall time: 6.95 ms


4999950000

In [10]:
%load_ext line_profiler

def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L
    return total

%lprun -f sum_of_lists sum_of_lists(5000)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-06 s

Total time: 0.008342 s
File: <ipython-input-10-206506eb5cdb>
Function: sum_of_lists at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def sum_of_lists(N):
     4         1          4.0      4.0      0.0      total = 0
     5         6         33.0      5.5      0.4      for i in range(5):
     6         5       7903.0   1580.6     94.7          L = [j ^ (j >> i) for j in range(N)]
     7         5        180.0     36.0      2.2          total += sum(L)
     8         5        222.0     44.4      2.7          del L # remove reference to L
     9         1          0.0      0.0      0.0      return total

С помощью профилирования производительности можно оценить, стоит ли оптимизация усилий и понять, какие блоки кода занимают больше всего времени. С них и стоит начинать оптимизацию.

## 2. Измеряйте объем потребляемой памяти

Измеряйте объем потребляемой памяти с помощью профайлеров памяти.

In [11]:
!pip install memory_profiler

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
  Using cached memory_profiler-0.58.0.tar.gz (36 kB)
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30189 sha256=29c50db2a887df5d034ded00c8ccfd4007ab45902c1e56af8ab134c1ab13c7b0
  Stored in directory: /home/leo/.cache/pip/wheels/6a/37/3e/d9e8ebaf73956a3ebd2ee41869444dbd2a702d7142bcf93c42
Successfully built memory-profiler
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [38]:
#%load_ext memory_profiler
%memit sum_of_lists(10**6)

peak memory: 96.66 MiB, increment: 30.79 MiB


Можно, конечно использовать и функцию `sys.getsizeof()`, однако она не возвращает суммарный объем для контейнерных объектов, а учесть все особенности распределения памяти в больших программах довольно проблематично.

## 3. Дизасемблируйте функции, методы, классы

С помощью модуля `dis` стандартной библиотеки можно посмотреть как функции и классы преобразуются в низкоуровневые инструкции интерпретатора. Так вы узнаете, какие атомарные инструкции выполняются в реальности и сможете взглянуть на процесс со стороны.

In [15]:
from dis import dis

In [19]:
def test_sum(a, b):
    return a+b

In [20]:
dis(test_sum)

  2           0 LOAD_FAST                0 (a)
              2 LOAD_FAST                1 (b)
              4 BINARY_ADD
              6 RETURN_VALUE
